In [21]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)

# 열 이름을 지정
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 

# horsepower 열의 누락 데이터('?') 삭제하고 실수형으로 변환
df['horsepower'] = df['horsepower'].replace('?', np.nan)      # '?'을 np.nan으로 변경
df = df.dropna(subset=['horsepower'], axis=0)                 # 누락데이터 행을 삭제
df['horsepower'] = df['horsepower'].astype('float')           # 문자열을 실수형으로 변환

# np.histogram 으로 3개의 bin으로 나누는 경계 값의 리스트 구하기
count, bin_dividers = np.histogram(df['horsepower'], bins=3)

# 3개의 bin에 이름 지정
bin_names = ['저출력', '보통출력', '고출력']

# pd.cut 으로 각 데이터를 3개의 bin에 할당
df['hp_bin'] = pd.cut(x=df['horsepower'],     # 데이터 배열
                      bins=bin_dividers,      # 경계 값 리스트
                      labels=bin_names,       # bin 이름
                      include_lowest=True)    # 첫 경계값 포함

# sklern 라이브러리 불러오기
from sklearn import preprocessing    

# 전처리를 위한 encoder 객체 만들기
label_encoder = preprocessing.LabelEncoder()       # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()     # one hot encoder 생성

# label encoder로 문자열 범주를 숫자형 범주로 변환
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))  
print(onehot_labeled)
print(type(onehot_labeled))

# 2차원 행렬로 형태 변경
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1) 
print(onehot_reshaped)
print(type(onehot_reshaped))

# 희소행렬로 변환
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
%pip install pandas numpy

In [26]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

# read_csv() 함수로 df 생성
df = pd.read_csv('./data/auto-mpg.csv', header=None)

# 열 이름을 지정
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
              'acceleration','model year','origin','name'] 
df.head(2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320


In [28]:
# 연비를 3개 구간으로 나눠서 새로운 컬럼명으로 추가
# 저연비, 보통, 고연비
# 연속형(수치형) -> 범주형(category)형 분리
df.mpg
df['mpg_qcut'] =  pd.qcut(df.mpg,q=3,labels=['저연비', '보통', '고연비'])
print(df.mpg_qcut.value_counts())
# 계급구간확인  라벨없이 적용후
temp = pd.qcut(df.mpg,q=3)
print(f'구간별 데이터 범위 : {temp.cat.categories}')

mpg_qcut
저연비    143
고연비    133
보통     122
Name: count, dtype: int64
구간별 데이터 범위 : IntervalIndex([(8.999, 19.0], (19.0, 26.933], (26.933, 46.6]], dtype='interval[float64, right]')


In [29]:
size, bins_range =  np.histogram(df.mpg,bins=np.array(3))
df['mpg_cut'] = pd.cut(df.mpg,bins=bins_range,labels=['저연비', '보통', '고연비'])
df.mpg_cut.value_counts()
print(f'계급별 구간 : {bins_range}')

계급별 구간 : [ 9.         21.53333333 34.06666667 46.6       ]


In [ ]:
pd.get_dummies(df.mpg_qcut,dtype=float)

In [ ]:
pd.get_dummies(df.mpg_cut,dtype=float)

In [ ]:
%pip install scikit-learn